In [16]:
import matplotlib.pyplot as plt
from scipy import stats
import xarray as xr
import pandas as pd
import numpy as np
import netCDF4
import random
import time

from sklearn.ensemble import RandomForestClassifier as skRF
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
import xgboost as xgb
import optuna



from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16

from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D



random.seed(42)

## THOUGHTS/IDEAS

8/19

- Capatability for multiple different DL models input. Right now the model is VGG16. Can change this in section 3b DL Training
- need to have the data put through train_test_split before fitting with the model. Evaluate performance. 


8/7 

- try something like [this article](https://towardsdatascience.com/how-to-cluster-images-based-on-visual-similarity-cd6e7209fe34) that clusters individual images into groups?


8/1:  
- Based on the 2D plots and general knowledge, do we want any variables that are NOT zero?
- There are A LOT of datapoints and its challenging to parse out the differences between each simulation. Try clustering to find more differences between the simulation?

## 1. Loading Data

In [2]:
def Simulation_Data_Extraction(file_path, DROP_VARS = None): 
    '''
    In:
        file_path (str): Path to the simulation data
        DROP_VARS (list): List of variables to drop when loading the Xarray simulation data
    Out:
        ds (Dataset): Xarray dataset
    '''
    ncfile= netCDF4.Dataset(file_path)
    ds = xr.open_dataset(xr.backends.NetCDF4DataStore(ncfile), drop_variables = DROP_VARS)
    return ds

Uncomment below the different length simulations

In [3]:
# # 36 hour simulation
# sim_dir =  "/discover/nobackup/jli30/scratch/notebooks"

# 156 hour simulation
sim_dir = "/discover/nobackup/alburke2/nu_WRF"

sim_file = "wrfout_d01_2021-03-13_12:00:00"
sim_44_file = f"{sim_dir}/lsw44_output/{sim_file}"
sim_55_file = f"{sim_dir}/lsw55_output/{sim_file}"
sim_77_file = f"{sim_dir}/lsw77_output/{sim_file}"

Include the desired variables in the "input_features" variable

In [4]:
ds_44_all = Simulation_Data_Extraction(sim_44_file) 
input_features = ['W', 'RE_HAIL_GSFC'] #, 'ACLHF']
data_vars = [i for i in ds_44_all.data_vars] 
drop_features = np.setdiff1d(data_vars,input_features)

Extract the different simulations

In [5]:
ds_44 = Simulation_Data_Extraction(sim_44_file,drop_features)

In [6]:
ds_55 = Simulation_Data_Extraction(sim_55_file,drop_features) 

In [7]:
ds_77 = Simulation_Data_Extraction(sim_77_file,drop_features)

### Plotting 2D data

In [ ]:
PLOT_HOUR = 100
PLOT_LEVEL = 0
VARIABLE = 'RE_HAIL_GSFC'

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(18, 8))

simulations = [
    ds_44[VARIABLE][PLOT_HOUR][PLOT_LEVEL],
    ds_55[VARIABLE][PLOT_HOUR][PLOT_LEVEL],
    ds_77[VARIABLE][PLOT_HOUR][PLOT_LEVEL],
    (ds_44[VARIABLE][PLOT_HOUR][PLOT_LEVEL]-ds_77[VARIABLE][PLOT_HOUR][PLOT_LEVEL]),
    (ds_44[VARIABLE][PLOT_HOUR][PLOT_LEVEL]-ds_55[VARIABLE][PLOT_HOUR][PLOT_LEVEL]),
    (ds_55[VARIABLE][PLOT_HOUR][PLOT_LEVEL]-ds_77[VARIABLE][PLOT_HOUR][PLOT_LEVEL])
]

simulation_names = ['44','55','77','44 v. 55', '44 v. 55', '55 v. 77']
s = 0
for row in np.arange(2):
    for col in np.arange(3):
        axs[row,col].set_xticks([])
        axs[row,col].set_yticks([])
        
        if row >= 1: plot_map = 'bwr'
        else: plot_map = 'viridis' 
        sim_plot = axs[row,col].imshow(simulations[s], cmap = plot_map)
        cbar = fig.colorbar(sim_plot, ax=axs[row,col])
        cbar.set_label(ds_44[VARIABLE].units)
        axs[row,col].set_title(f'{simulation_names[s]}')
        s+=1
        
plt.suptitle(f'Variable: {VARIABLE}, Hour: {PLOT_HOUR}, Level: {PLOT_LEVEL}',fontsize=15)
plt.show()
plt.close()

## 2. Data Processing

### 2a. Machine learning

In [ ]:
def Simulation_Data_Preprocessing(input_data, sim_name, NUM_LEVEL = 40, START_HOUR = 0, END_HOUR = 36, RAND_DATAP = 0.1):
    '''
    In:
        input_data (Dataset): input simulation data
        sim_name (str): Name of the simulation. Either '44','55', or '77' for this work
        NUM_LEVEL (int): Number of levels desired for a given hour
        START_HOUR (int): The first hour input to a list process
        END_HOUR (int): The last hour input to a list process
        RAND_DATAP (float < 1.0): A float value describing what percentage of the total datapoints is used
    Out:
        vars_df (DataFrame): 1D chosen datapoints for each variable as a pandas Dataframe
        vars_df_col_names (list): List of the different names of each saved variable. May include the specific
                                hour or level chosen for a given variable
    '''
    print(f'Simulation: {sim_name}, Hours: {START_HOUR}-{END_HOUR}')
    HOURS=np.arange(START_HOUR,END_HOUR)
    
    # Create a dictionary with the chosen varaibles as the keys
    vars_df_dict = dict.fromkeys(ds_44.data_vars,[])
    vars_df_col_names = [] 
    
    for var in ds_44.data_vars:
        ################ (SEE CELL BELOW) 
        # add a step that looks for the mean and std of the chosen hours?
        # Only save the datapoints that are outside the 1st sd 
        # mean, std = Scaling(
        #      np.ravel(input_data[var][HOURS]).reshape(-1, 1)
        #             )
        ################
        print(f'\nVariable: {var}')
        all_hour_var = []
        for hour in HOURS: 
            if hour%5 == 0: print(f'Hour: {hour}')
            new_var = f'{sim_name}_{var}_h{hour}'
            if np.ndim(input_data[var][hour]) > 2:
                #data are 3D, stack the different levels into different "vars"
                level_list = np.arange(len(input_data[var][hour]))
                #randomly choose which levels to use
                rand_levels = random.choices(level_list,k=NUM_LEVEL)
                for level in rand_levels:
                    hourly_sim_level = input_data[var][hour][level].values.ravel()
                    all_hour_var.append(hourly_sim_level)
                    #Save the hourly/level variable data
                    hourly_level_name = f'{new_var}_l{level}'
                    vars_df_col_names.append(hourly_level_name)
            else:
                #data are 2D, no extra levels
                hourly_sim = input_data[var][hour].values.ravel()
                vars_df_col_names.append(new_var)
                all_hour_var.append(hourly_sim)
        #Save the total hourly/level variable data into one variable        
        vars_df_dict[var] = np.ravel(all_hour_var)
    #Convert the dictionary to a dataframe
    vars_df = pd.DataFrame.from_dict(vars_df_dict)  
    
    #Remove any hail upflux values == 0
    vars_df = vars_df[(vars_df['RE_HAIL_GSFC']!=0)]
    
    print(f'\n--------- Processed Simulation {sim_name} Data ---------')
    print(vars_df)
    
    return vars_df, vars_df_col_names

In [ ]:
# def Scaling(input_data, SIM_NAME = None, SD_FACTOR = 1.0):
#     '''
#     (Not currently in use but for possible future) 
#     In:
#         input_data (Dataset): input simulation data
#         SIM_NAME (str): Name of the simulation. Either '44','55', or '77' for this work
#         SD_FACTOR (float): The standard deviation from the mean desired. 
#             So, 1 sd from the mean, or 2 sd from the mean, etc. 
#     Out:
#         upper_limit (float): Upper limit value greater than the SD_FACTOR from the mean
#         lower_limit (float): Lower limit value greater than the SD_FACTOR from the mean

#     '''
#     start_time = time.time()
#     scaler = StandardScaler()
#     scaler.fit(input_data)
#     scaler_mean = scaler.mean_
#     scaler_std = np.sqrt(scaler.var_)
#     print("--- %s seconds ---" % (time.time() - start_time))
#     print(SIM_NAME,'Mean',scaler_mean,'Sd',scaler_std)
#     upper_limit = scaler_mean + (SD_FACTOR*scaler_std)
#     lower_limit = scaler_mean - (SD_FACTOR*scaler_std)
#     return upper_limit,lower_limit

In [ ]:
start = 0
end = 150
processing_params = {
    'START_HOUR':start,
    'END_HOUR':end, 
    'NUM_LEVEL':10
         }

Change the starting/ending hours depending on the length of the simulation. 
Change the number of levels ("NUM_LEVEL") randomly chosen per desired hour range

In [ ]:
df_44_vars, df_44_image_names = Simulation_Data_Preprocessing(ds_44, '44', **processing_params)

In [ ]:
df_55_vars, df_55_image_names = Simulation_Data_Preprocessing(ds_55, '55', **processing_params)

In [ ]:
df_77_vars, df_77_image_names = Simulation_Data_Preprocessing(ds_77, '77', **processing_params)

#### Clustering

In [ ]:
cluster_number = 5

km_params  = {
    'n_clusters':cluster_number, 
    'n_init':10, 
    'random_state':42
    }

In [ ]:
# gm_pred = GaussianMixture(random_state=42).fit_predict(df_44_vars)

In [ ]:
%%time
km_44_pred = KMeans(**km_params).fit_predict(df_44_vars)

In [ ]:
%%time
km_55_pred = KMeans(**km_params).fit_predict(df_55_vars)

In [ ]:
%%time
km_77_pred = KMeans(**km_params).fit_predict(df_77_vars)

#### Stratified Cluster Sampling

Smallest cluster between 44,55,77

Randomly choose the same number of the smallest cluster across the different clusters 

In [ ]:
def Cluster_Counts(kmeans_output):
    '''
    In:
        kmeans_output (list): Kmeans output of the clusters assigned to each given datapoint
    Out:
       min_count (int): The minimum number of datapoints across the different clusters
    '''
    unique, counts = np.unique(kmeans_output, return_counts=True)
    min_count = np.nanmin(counts)
    print(min_count)
    return min_count

def Stratified_Cluster_Sampling(min_count,kmeans_output):
    '''
    In:
        min_count (int): The minimum number of datapoints desired
        kmeans_output (list): Kmeans output of the clusters assigned to each given datapoint
    Out:
       stratified_clusters (Array): Clusters with the same chosen min_count of datapoints
    '''
    stratified_clusters = np.array([])
    for cluster in np.arange(np.unique(kmeans_output)):
        print(f'Cluster {cluster}')  
        cluster_data = np.where(kmeans_output == cluster)[0]
        sample_data = np.random.choice(cluster_data,min_count,replace=False)
        stratified_clusters = np.append(stratified_clusters, sample_data)
    stratified_clusters = stratified_clusters.astype('int')
    print(len(stratified_clusters))
    return stratified_clusters

In [ ]:
COUNT_44 = Cluster_Counts(km_44_pred)
COUNT_55 = Cluster_Counts(km_55_pred)
COUNT_77 = Cluster_Counts(km_77_pred)
#Needs the smallest cluster size between the 44, 55, 77 labels 
SMALLEST_COUNT = np.nanmin([COUNT_44,COUNT_55,COUNT_77])
print('Smallest cluster count:', SMALLEST_COUNT)


In [ ]:
CLUSTER_44 = Stratified_Cluster_Sampling(SMALLEST_COUNT,km_44_pred)
CLUSTER_55 = Stratified_Cluster_Sampling(SMALLEST_COUNT,km_55_pred)
CLUSTER_77 = Stratified_Cluster_Sampling(SMALLEST_COUNT,km_77_pred)

#### Plots

In [ ]:
num_dp = 1000000
COLUMNS = 3
data_sim_list = [df_44_vars,df_55_vars,df_77_vars] #, df_44_vars])
kmean_sim_list = [km_44_pred,km_55_pred,km_77_pred] #, gm_pred])

fig, axs = plt.subplots(nrows=1, ncols=COLUMNS, figsize=(20, 5))

for col in np.arange(COLUMNS):
    sim = data_sim_list[col]
    print(sim.shape)

    axs[col].scatter(sim.iloc[:num_dp,0],sim.iloc[:num_dp,1],c=kmean_sim_list[col][:num_dp])
    axs[col].set_xlabel(ds_44['W'].units)
    axs[col].set_ylabel(ds_44['RE_HAIL_GSFC'].units)
    
plt.suptitle(f'{num_lev} rand lev. Hours: {start} - {end}',fontsize=20)
plt.show()

In [ ]:
num_dp = 100000000
COLUMNS = 3

data_sim_list = [df_44_vars, df_55_vars, df_77_vars]
data_cluster_list = [df_44_vars.iloc[CLUSTER_44,:],df_55_vars.iloc[CLUSTER_55,:],df_77_vars.iloc[CLUSTER_77,:]]

fig, axs = plt.subplots(nrows=1, ncols=COLUMNS, figsize=(20, 5))

for c in np.arange(COLUMNS):
    sim = data_sim_list[c]
    print(sim.shape)
    axs[c].scatter(sim.iloc[:num_dp,0],sim.iloc[:num_dp,1], color='blue',s=1)
    
    clus = data_cluster_list[c]
    print(clus.shape)
    axs[c].scatter(clus.iloc[:num_dp,0],clus.iloc[:num_dp,1], color='red',s=1)
    
    axs[c].set_xlabel(ds_44['W'].units)
    axs[c].set_ylabel(ds_44['RE_HAIL_GSFC'].units)
    
plt.suptitle(f'Cluster (red) v. Random Sample (blue) \n Hours: {start} - {end}',fontsize=20)
plt.show()

Frequency histograms for the total and cluster data

In [ ]:
cols = df_44_vars.columns
print(cols)

num_dp = 100000000
COLUMNS = 3



data_cluster_list = [df_44_vars.iloc[CLUSTER_44,:],
                    df_55_vars.iloc[CLUSTER_55,:],
                    df_77_vars.iloc[CLUSTER_77,:]]

label_list = np.array(['44','55','77'])

fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(20, 5))

for c in np.arange(4): 
    if c%2 > 0 : 
        axs[c].set_xlabel(ds_44['RE_HAIL_GSFC'].units)
        if c > 1: 
            axs[c].set_title('TOTAL RE_HAIL_GSFC')
        else: 
            axs[c].set_title('CLUSTER RE_HAIL_GSFC')
    else:
        axs[c].set_xlabel(ds_44['W'].units)
        if c > 1: 
            axs[c].set_title('TOTAL W')
        else: 
            axs[c].set_title('CLUSTER W')
    
    for sim in np.arange(3):
        if c < 2: 
            cluster_var = data_cluster_list[sim].iloc[:,c%2]
            print(np.shape(cluster_var))
            axs[c].hist(cluster_var, label=label_list[sim], log=True, histtype='step') #,linewidth=2)
        else: 
            sim_var = data_sim_list[sim].iloc[:,c%2]
            print(np.shape(sim_var))
            axs[c].hist(sim_var, label=label_list[sim], log=True, histtype='step') #,linewidth=2)
        
    axs[0].set_ylabel('Frequency')

    
plt.legend()
plt.suptitle(f'Hours: {start} - {end}',fontsize=16)
plt.show()

 #### Combining Data

All randomly sampled simulation data

In [ ]:
sim_44_55_77 = pd.concat([df_44_vars,df_55_vars,df_77_vars])

label_sim = np.hstack((
    np.full(len(df_44_vars),0),
    np.full(len(df_44_vars),1),
    np.full(len(df_44_vars),2))
)

Even balance cluster data

In [ ]:
clus_44_55_77 = pd.concat([df_44_vars.iloc[CLUSTER_44,:], df_55_vars.iloc[CLUSTER_55,:],df_77_vars.iloc[CLUSTER_77,:]])

label_clus = np.hstack((
    np.full(len(CLUSTER_44),0),
    np.full(len(CLUSTER_44),1),
    np.full(len(CLUSTER_44),2))
)

In [ ]:
print('All sampled pts')
print(sim_44_55_77, label_sim)
print()
print('Clustered pts')
print(clus_44_55_77, label_clus)

### 2b. Deep learning

In [8]:
def Simulation_Data_Preprocessing_2D(input_data, sim_name, input_features): #, NUM_LEVEL = 40, START_HOUR = 0, END_HOUR = 36, RAND_DATAP = 0.1):
    '''
    In:
        input_data (Dataset): input simulation data
        sim_name (str): Name of the simulation. Either '44','55', or '77' for this work
        input_features (list): The chosen input variables
    Out:
        vars_2D_list (list): 4D data (variable, examples, x, y) 
    '''
    print(f'Simulation: {sim_name}, Features: {input_features}')
    
    #W has 60 levels and the hail parameter has 61. To keep consistency, only use 60
    desired_levels = 60 
    vars_2D_list = []
    for var in ds_44.data_vars:
        new = input_data[var].values
        time,level,x,y = new.shape
        if level != desired_levels:
            new = new[:,:desired_levels,:,:]
        print(var, new.shape)
        vars_2D_list.append(new.reshape([time * desired_levels, x, y]))
    print(f'Total {np.shape(vars_2D_list)}')
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.33, random_state=42)

    
    return vars_2D_list

In [9]:
list2D_44 = Simulation_Data_Preprocessing_2D(ds_44, '44', input_features)

Simulation: 44, Features: ['W', 'RE_HAIL_GSFC']
W (151, 60, 240, 300)
RE_HAIL_GSFC (151, 60, 240, 300)
Total (2, 9060, 240, 300)


In [10]:
list2D_55 = Simulation_Data_Preprocessing_2D(ds_55, '55', input_features)

Simulation: 55, Features: ['W', 'RE_HAIL_GSFC']
W (151, 60, 240, 300)
RE_HAIL_GSFC (151, 60, 240, 300)
Total (2, 9060, 240, 300)


In [11]:
list2D_77 = Simulation_Data_Preprocessing_2D(ds_77, '77', input_features)

Simulation: 77, Features: ['W', 'RE_HAIL_GSFC']
W (151, 60, 240, 300)
RE_HAIL_GSFC (151, 60, 240, 300)
Total (2, 9060, 240, 300)


Combining Data

Want: (Num. Vars, All examples, x, y) 

In [28]:
combined2D_predictors = np.hstack([list2D_44,list2D_55,list2D_77])
print(np.shape(combined2D_predictors))
DL_predictors = np.moveaxis(combined2D_predictors,0,-1)
print(np.shape(DL_predictors))

(2, 27180, 240, 300)
(27180, 240, 300, 2)


In [29]:
print(DL_predictors.shape[1:])

(240, 300, 2)


In [23]:
#Number of tiles 
num_examples = np.shape(list2D_77)[1]
combined_labels = ['44']*num_examples+['55']*num_examples+['77']*num_examples
print(np.shape(combined_labels))

(27180,)


## 3. AI Setup

### 3a. Machine Learning

#### Random Forest

In [ ]:
rf_search_space={
    "n_estimators": [75, 100, 125, 150, 175, 200, 250, 300, 400, 500],
    "max_depth" : [5,10, 30, 50, 80, 90, 100, 110],
    "min_samples_leaf" : [1, 2, 3, 4, 5],
    "min_samples_split" : [2, 4, 8, 10],
    "bootstrap" : [True, False],
    "max_features" : ['auto', 'sqrt', 'log2'],
    
}

list_trees = [75, 100, 125, 150, 175, 200, 250, 300, 400, 500]
max_depth = [5, 10, 30, 50, 80, 90, 100, 110]
min_samples_leaf = [1, 2, 3, 4, 5]
min_samples_split = [2, 4, 8, 10]
bootstrap = [True, False]
max_features = ['auto', 'sqrt', 'log2']

TREES_AND_DEPTH_ONLY = False
GRID_SEARCH = True

def cpu_rf_objective(trial, X, y):
    param = {'n_estimators': trial.suggest_categorical('n_estimators', list_trees), 
           'max_depth':trial.suggest_categorical('max_depth', max_depth), 
           'min_samples_split':trial.suggest_categorical('min_samples_split', min_samples_split), 
           'min_samples_leaf':trial.suggest_categorical('min_samples_leaf', min_samples_leaf), 
           'bootstrap': trial.suggest_categorical('bootstrap', bootstrap),
           'criterion':'gini', 
           #'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 1e-8, 1.0, log=True), 
           'max_features':trial.suggest_categorical('max_features', max_features), 
           'max_leaf_nodes':None, 
           'min_impurity_decrease':0.0, 
           'oob_score':False, 
           'n_jobs':-1, 
           # 'random_state':42, 
           'verbose':0, 
           'warm_start':False, 
           'class_weight':None, 
           'ccp_alpha':0.0, 
           'max_samples':None
        }
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = np.empty(5)
    
    for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        try:
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        except:
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
        
        model = skRF(**param)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        cv_scores[idx] = f1_score(y_val, preds, average='weighted') 
        #using weighted for the multiclass because each class is balanced but shuffled
        
        if cv_scores[idx] == 0.0:
            print('Pruning because of 0.0 score.')
            return 0.0
        print('Fold {}: {}'.format(idx, cv_scores[idx]))
    return np.mean(cv_scores)


#### XGBoost

In [ ]:
xgb_search_space={
    "n_estimators":[50, 75, 100, 150, 175, 200, 250, 300, 500],
    "max_depth":[30, 35, 40, 45, 50, 75],
    "booster": ["gbtree", "dart"]
}
#     "n_estimators": [75, 100, 125, 150, 175, 200, 250, 300, 400, 500],
#     "max_depth" : [5, 10, 30, 50, 80, 90, 100, 110],
#     "min_samples_leaf" : [1, 2, 3, 4, 5],
#     "min_samples_split" : [2, 4, 8, 10],
#     "bootstrap" : [True, False],
#     "max_features" : ['auto', 'sqrt', 'log2'],
    

def cpu_xgb_objective(trial, X, y):
    xbg_param = {
    "verbosity": 0,
    "objective": "binary:logistic",
    "tree_method": "hist",
    "n_jobs": -1,
    "n_estimators": trial.suggest_categorical("n_estimators", [50, 75, 100, 150, 175, 200, 250, 300, 500]),
    "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
    "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
    "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    "subsample": trial.suggest_float("subsample", 0.2, 1.0), 
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),   
    "max_depth":  trial.suggest_int("max_depth", 30, 50, step=5),     
    # "min_child_weight":  trial.suggest_int("min_child_weight", 2, 10), 
    # "eta":  trial.suggest_float("eta", 1e-8, 1.0, log=True), 
    # "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True), 
    # "grow_policy":  trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
    }
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = np.empty(5)
    
    for idx, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        try:
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        except:
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
        
        pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation_0-auc")
        classifier = xgb.XGBClassifier(**xbg_param)
        eval_set = [(X_train, y_train), (X_val, y_val)]
        eval_metric = ["error","auc"]
        classifier.fit(X_train, 
                   y_train, 
                   eval_set=eval_set, 
                   eval_metric=eval_metric, 
                   early_stopping_rounds=10, 
                   callbacks=[pruning_callback],
                   verbose=False)

        preds = model.predict(X_val)
        cv_scores[idx] = f1_score(y_val, preds, average='weighted') 
        #using weighted for the multiclass because each class is balanced but shuffled
        
        if cv_scores[idx] == 0.0:
            print('Pruning because of 0.0 score.')
            return 0.0
        print('Fold {}: {}'.format(idx, cv_scores[idx]))
    return np.mean(cv_scores)

In [ ]:
def tuning(obj, X_chosen, y_chosen, search_space, #TEST_TRAIN_RATIO = 0.3, 
           NN = False, NTRIALS = 2):
    
    # Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    #     X_chosen, y_chosen, test_size=TEST_TRAIN_RATIO, random_state=42)
    
    start_time = time.time()
    optuna.logging.set_verbosity(optuna.logging.INFO)
    if GRID_SEARCH:
        study = optuna.create_study(study_name='RF Tuning Grid Search', 
                                    direction='maximize',
                                    sampler=optuna.samplers.GridSampler(search_space))
    else:
        study = optuna.create_study(study_name='RF Tuning',
                                    direction='maximize')
        
    study.optimize(lambda trial: obj(trial, X_chosen, y_chosen), n_trials=NTRIALS, timeout=30*600)
    print("--- %s seconds ---" % (time.time() - start_time))
   
     #############################
    trials = study.best_trials            
    max_trial_score = max([trial.values[0] for trial in trials])
    max_trial_params = [trial.params for trial in trials 
                        if trial.values[0] == max_trial_score][0]
    max_trial_params['n_jobs'] = -1
    score_print = int(np.round(max_trial_score,4)*1000)
    print('\nMax score:', score_print)
    # score_out_file = f'{out_file}_MaxScore{score_print}.pkl'
    # print(score_out_file)
    
    #############################
    #Testing
    start_time = time.time()
    hyperparameters = max_trial_params
    hyperparameters['n_jobs'] = -1
    print('Using these params:')
    print(hyperparameters)
    # tuned_classifier = skRF(**hyperparameters)
    # tuned_classifier.fit(X_chosen, y_chosen)
    print("--- %s seconds ---" % (time.time() - start_time))

#### Tuning/Training

Which dataset to use first?

large sample: sim_44_55_77, label_sim
clustered sample: clus_44_55_77, label_clus


def tuning(partial_obj, X_chosen, y_chosen, TEST_TRAIN_RATIO = 0.3, NN = False, NTRIALS = 2)

In [ ]:
# tuning(cpu_xgb_objective,clus_44_55_77,label_clus, search_space = xgb_search_space, NTRIALS = 2)

In [ ]:
tuning(cpu_rf_objective,clus_44_55_77,label_clus, rf_search_space, NTRIALS = 10)

In [ ]:
tuning(cpu_xgb_objective,clus_44_55_77,label_clus, NTRIALS = 2)

### 3b. DL Training

- predictors: (examples, x, y, variables)
- labels: (examples, 3) multiclass classification so 001/010/100

In [24]:
lb = LabelBinarizer()
multiclass_labels = lb.fit_transform(combined_labels)
print(multiclass_labels.shape)
print(multiclass_labels)

(27180, 3)
[[1 0 0]
 [1 0 0]
 [1 0 0]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [32]:
# create the base model, no pre-trained weights
dl_model = VGG16(weights=None, include_top=True, classes=3, input_shape = DL_predictors.shape[1:])
dl_model.compile(optimizer='Adam', loss='categorical_crossentropy')
print(dl_model.summary())
dl_model.fit(
    DL_predictors,
    multiclass_labels,
    batch_size=128,
    epochs=3,
    validation_split=0.33)

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 240, 300, 2)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 240, 300, 64)      1216      
                                                                 
 block1_conv2 (Conv2D)       (None, 240, 300, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 120, 150, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 120, 150, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 120, 150, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 60, 75, 128)       0     

## 4. Previous Empirical Trials

choosing the 3D levels by the frequency of data

In [ ]:
# def calculating_frequency(input_data,round_by=3):
#     '''
#     input_data (array) : input simulation data, assumed a single variable
#     round_by (int): how many decimals are wanted when calculating the frequency of given floats
#     '''
    
#     round_input_1D = np.round(np.ravel(input_data),round_by)
#     unique, counts = np.unique(round_input_1D, return_counts=True)
#     unique_counts_2D = np.asarray((unique, counts)).T
    
#     max_freq_ind = np.argmax(unique_counts_2D[:,1])
#     max_value_freq = unique_counts_2D[max_freq_ind][1]
    
#     min_freq_ind = np.argmin(unique_counts_2D[:,1])
#     min_freq_value = unique_counts_2D[min_freq_ind][0]
   
#     return min_freq_value, max_value_freq



# def preprocessing(input_sim, sim_name, START_HOUR = 0, END_HOUR = 36):
#     print(f'Simulation: {sim_name}, Hours: {START_HOUR}-{END_HOUR}')
#     HOURS=np.arange(START_HOUR,END_HOUR)
#     vars_df_dict = dict.fromkeys(ds_44.data_vars,[])
#     vars_df_col_names = [] 
#     for hour in HOURS: 
#         print(f'Hour: {hour}')
#         for var in ds_44.data_vars:
#             new_var = f'{sim_name}_{var}_h{hour}'
#             if np.ndim(input_sim[var][hour]) > 2:
#                 #data are 3D, stack the different levels into different "vars"
#                 # Found that the 2 images that show the most change are the level
#                 # with the most frequency of the max value and
#                 # level with lowset value of the minimum frequency
#                 ######################
#                 #minfv aka minimum frequency value 
#                 minimum_freq_value = 100.0
#                 minfv_level = None
#                 minfv_var_name = None
#                 ######################
#                 #maxvf aka maximum value frequency
#                 maximum_value_freq = -100.0
#                 maxvf_level = None
#                 maxvf_var_name = None
#                 ######################
#                 num_levels = np.shape(input_sim[var][hour])[0]
#                 for level in np.arange(num_levels):
#                     hourly_sim_level = input_sim[var][hour][level].values
#                     hourly_sim_shape = hourly_sim_level.shape[0]*hourly_sim_level.shape[1]
#                     hourly_lev_minfv, hourly_lev_maxvf = freq_stuff(hourly_sim_level,sim_name)
#                     if (hourly_lev_minfv < minimum_freq_value) & (hourly_lev_minfv != 0): 
#                         #Dont want the minimum value to be 0 when the values may only start at 0
#                         minimum_freq_value = hourly_lev_minfv
#                         minfv_level = level
#                         minfv_var_name = f'{new_var}_l{level}_minfv'
                    
#                     if (hourly_lev_maxvf > maximum_value_freq) & (hourly_lev_maxvf != hourly_sim_shape): 
#                         # Dont want the maximum frequency to just pick the image with all the same value aka blank image
#                         maximum_value_freq = hourly_lev_maxvf
#                         maxvf_level = level 
#                         maxvf_var_name = f'{new_var}_l{level}_maxvf' 
#                 ######################
#                 hourly_minfv = input_sim[var][hour][minfv_level].values.ravel()
#                 vars_df_col_names.append(minfv_var_name)   
#                 vars_df_dict[var].extend(hourly_minfv)
#                 ######################
#                 hourly_maxvf = input_sim[var][hour][maxvf_level].values.ravel()
#                 vars_df_col_names.append(maxvf_var_name)
#                 vars_df_dict[var].extend(hourly_maxvf)
#                 ######################
#             else:
#                 #data are 2D, no extra levels
#                 hourly_sim = input_sim[var][hour].values.ravel()
#                 # vars_df_list_data.append(hourly_sim)
#                 vars_df_col_names.append(new_var)
#                 vars_df_dict[var].extend(hourly_sim)
                
#     vars_df = pd.DataFrame.from_dict(vars_df_dict)
#     vars_df['label'] = np.full(len(vars_df.iloc[:,0]),sim_name)     
#     print(vars_df)
    
#     return vars_df, vars_df_col_names